## Problema Final - Algoritmo Genético

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
# Load the data from Excel
file = 'https://github.com/KanonStarbringer/Trabalho-Final-Modelagem-com-PL-PI/raw/main/trabalho%20final.xlsx'
df = pd.read_excel(file, sheet_name='Instancia 1')

In [3]:
# Extract data
## Extraindo os parametros do DataFrame
n = df.shape[1] - 1
r = df.iloc[0, 1:].tolist()
d = df.iloc[1, 1:].tolist()
p = df.iloc[2, 1:].tolist()
w = df.iloc[3, 1:].tolist()
tipos = df.iloc[4, 1:].tolist()

In [4]:
# Number of jobs
n = len(r)
T = max(p) + sum(d)  # Planning horizon

In [5]:
# Evaluate fitness function
def evaluate(schedule):
    total_cost = 0
    end_times = np.zeros(n)
    for i in range(n):
        job = schedule[i]
        start_time = max(r[job], end_times[i-1] if i > 0 else 0)
        end_time = start_time + d[job]
        delay = max(0, end_time - p[job])
        penalty = 0
        if delay > 0:
            if delay <= 5:
                penalty = w[job] * delay
            else:
                penalty = w[job] * 5 + w[job] * 2 * (delay - 5)
        total_cost += penalty
        end_times[i] = end_time
        
        # Machine changeover
        if i > 0 and tipos[schedule[i-1]] != tipos[job]:
            start_time += 1
            end_time += 1
            end_times[i] += 1
            
    return total_cost


In [6]:
# Generate initial population
def generate_initial_population(pop_size):
    population = []
    for _ in range(pop_size):
        individual = list(np.random.permutation(n))
        population.append(individual)
    return population

In [7]:
# Selection
def selection(population, fitnesses):
    selected = random.choices(population, weights=fitnesses, k=len(population))
    return selected

In [8]:
# Crossover
def crossover(parent1, parent2):
    point = random.randint(1, n-2)
    child1 = parent1[:point] + [gene for gene in parent2 if gene not in parent1[:point]]
    child2 = parent2[:point] + [gene for gene in parent1 if gene not in parent2[:point]]
    return child1, child2

In [9]:
# Mutation
def mutation(individual):
    point1, point2 = random.sample(range(n), 2)
    individual[point1], individual[point2] = individual[point2], individual[point1]
    return individual

In [10]:
# Genetic Algorithm
def genetic_algorithm(pop_size, generations):
    population = generate_initial_population(pop_size)
    for generation in range(generations):
        fitnesses = [1/evaluate(individual) for individual in population]
        population = selection(population, fitnesses)
        next_population = []
        for i in range(0, pop_size, 2):
            parent1, parent2 = population[i], population[i+1]
            child1, child2 = crossover(parent1, parent2)
            next_population.append(mutation(child1))
            next_population.append(mutation(child2))
        population = next_population
    best_individual = min(population, key=evaluate)
    return best_individual, evaluate(best_individual)

In [11]:
# Run the algorithm
best_schedule, best_cost = genetic_algorithm(pop_size=100, generations=1000)
print(f"Best schedule: {best_schedule}")
print(f"Best cost: {best_cost}")

Best schedule: [3, 2, 4, 5, 1, 0, 6]
Best cost: 83.0
